# **A Quick Data Wrangling Tutorial**
> ## *Author* : [Rathachai CHAWUTHAI](https://rathachai.creatier.pro/) , Ph.D
> ### *Affiliation* : Computer Engineering, King Mongkut's Institute of Technology Ladkrabang (KMITL)
> #### *Updated Date* : 2021-08-18
---

> <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.



---

## About Datasets

**Customer Churn Datasets**

https://docs.google.com/spreadsheets/d/e/2PACX-1vQxcCDTAalMyu4Tc_eelskRhZXldH9TSliZLFcW39_LlzOkIReCRdK0V7JR6APjJH-EluveA5c-m237/pubhtml

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib_inline as plt


## Load Datasets

In [ ]:
cust_csv_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQxcCDTAalMyu4Tc_eelskRhZXldH9TSliZLFcW39_LlzOkIReCRdK0V7JR6APjJH-EluveA5c-m237/pub?gid=0&single=true&output=csv"
churn_csv_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQxcCDTAalMyu4Tc_eelskRhZXldH9TSliZLFcW39_LlzOkIReCRdK0V7JR6APjJH-EluveA5c-m237/pub?gid=755674277&single=true&output=csv"

In [ ]:
cust_df = pd.read_csv(cust_csv_path)

In [ ]:
churn_df = pd.read_csv(churn_csv_path)

In [ ]:
cust_df

In [ ]:
churn_df

## Merge Datasets

In [ ]:
df = pd.merge(cust_df, churn_df, how="left", on="cust_id")

In [ ]:
df

## Drop Missing Values

In [ ]:
df.info()

In [ ]:
df.dropna(subset=["churn"], inplace=True)

In [ ]:
df

In [ ]:
df.info()

## Cap the Outliners

In [ ]:
%matplotlib
df.boxplot(column="height")

In [ ]:
df.dropna().sort_values('height')[['height']].plot.bar(use_index=False, figsize=(20,3), xticks=[], xlim=(0,1200), ylim=(100,500))

In [ ]:
low, high = df["height"].quantile([0.05, 0.95])
print(low, high)

In [ ]:
df.loc[df[df["height"]<low].index,"height"]=low

In [ ]:
df.loc[df[df["height"]>high].index,"height"]=high

In [ ]:
df.boxplot(column="height")

## Infer Missing Values (I)



In [ ]:
df.info()

In [ ]:
df.sort_values("height")

In [ ]:
df["height"].fillna(0)

In [ ]:
df["height"].fillna(df["height"].mean())

In [ ]:
df.boxplot(column="height")

In [ ]:
df.boxplot(column="height", by="gender")

In [ ]:
# Recommended !!!!
df['height'].fillna(df.groupby('gender')['height'].transform('mean'))

In [ ]:
df['height'] = df['height'].fillna(df.groupby('gender')['height'].transform('mean'))

In [ ]:
df.info()

## Extract Date Time

In [ ]:
df["birth"]

In [ ]:
df["birth"] = pd.to_datetime(df["birth"], format='%m/%d/%Y')

In [ ]:
df["birth"]

In [ ]:
df["birth"].dt.year

In [ ]:
2022 - df["birth"].dt.year

In [ ]:
df["age"] = 2022 - df["birth"].dt.year

In [ ]:
df

## Bining

In [ ]:
df["age"].hist()

In [ ]:
cut_labels = ['young', 'teen', 'adult', 'senior']
cut_bins = [0, 13, 25, 61, 100]
df['age_group'] = pd.cut(df['age'], bins=cut_bins, labels=cut_labels)

In [ ]:
df

In [ ]:
df.boxplot(column="age", by="age_group")

## Infer Missing Values (II)

In [ ]:
df.info()

In [ ]:
df.boxplot(column="age", by="job")

In [ ]:
ser = pd.Series(['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd'])
ser.mode()

In [ ]:
def f_mode(dat):
  if len(dat)>0:
    return dat.mode().iloc[0]
  else:
    return ""

In [ ]:
f_mode(ser)

In [ ]:
df['job'].fillna(df.groupby('age_group')['job'].transform(f_mode))

In [ ]:
df['job'] = df['job'].fillna(df.groupby('age_group')['job'].transform(f_mode))

In [ ]:
df.info()

In [ ]:
df.boxplot(column="age", by="job")

## Infer Missing Values (III)

In [ ]:
df.info()

In [ ]:
df.plot.scatter(x="age", y="salary")

In [ ]:
df.sort_values("age", inplace=True)

In [ ]:
df["salary"] = df["salary"].interpolate()

In [ ]:
df.info()

In [ ]:
df.plot.scatter(x="age", y="salary")

## Log Scale

In [ ]:
df[df['job']=='worker'].plot.scatter(x='age',y='salary')

In [ ]:
df["salary_log"] = df["salary"].apply(np.log)

In [ ]:
df[df['job']=='worker'].plot.scatter(x='age',y='salary_log')

## Binary Encoding

In [ ]:
df[["gender"]]

In [ ]:
df["gender"].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lb_make = LabelEncoder()

In [ ]:
df["gender_code"] = lb_make.fit_transform(df["gender"])

In [ ]:
df

## One-Hot Encoding

In [ ]:
df[["payment"]]

In [ ]:
df['payment'].unique()

In [ ]:
df = pd.get_dummies(df, columns=["payment"])

In [ ]:
df

In [ ]:
df[["payment_cash", "payment_cheque", "payment_credit"]]

## Text Extraction

In [ ]:
df[["address"]]

In [ ]:
"14/24, ปทุมวัน, กรุงเทพ".split(", ")

In [ ]:
"14/24, ปทุมวัน, กรุงเทพ".split(", ")[-1]

In [ ]:
df["address"].str.split(", ").str[-1]

In [ ]:
df["city"] = df["address"].str.split(", ").str[-1]

In [ ]:
df



---
https://rathachai.creatier.pro/

## つづく